This is our notebook for 6.853 project. Our goal is to reduce mode-collapse when training GANs. We aim to do this by using existing literature on multiplayer, zero-sum games to phrase the GAN problem as a polymatrix game. Then, we aim to use the reduction to the high-stakes lawyer game. 

Credit for portions of the code to Github user znxlwm.

In [2]:
import torch
import torchvision
import torchvision.datasets as datasets
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import scipy.misc
import pickle

In [3]:
#IMPORTS FUNCTIONS FROM UTIL
%run Utils.py

We first load the MNIST dataset and normalize

In [4]:
input_size = 28
batch_size = 100

trans = transforms.Compose(
            [transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean = (0.1307, ), std = (0.3081,))
            ])
                
data_loader = torch.utils.data.DataLoader(
               datasets.MNIST(root = './data', train=True, download = True, transform = trans), 
               batch_size = batch_size, 
               shuffle = True)

In [5]:
d = data_loader.__iter__().__next__()[0]

In [6]:
#create dataloader for pytorch
data_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size = 100, shuffle = True)
num_batches = len(data_loader)

NameError: name 'mnist_trainset' is not defined

In [7]:
#Next step: make use convolutions isntead of flattened image.
class DiscriminatorNet(torch.nn.Module):
    #Discriminator class. Outputs whehter it thinks the image is original or not.
    def __init__(self, in_dim, out_dim, input_size):
        super(DiscriminatorNet, self).__init__()
        self.in_dim = in_dim 
        self.out_dim = out_dim
        self.input_size = input_size

        self.conv = nn.Sequential(
            nn.Conv2d(self.in_dim, 64, kernel_size = 4, stride = 2, padding = 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size = 4, stride = 2, padding = 1),
            nn.BatchNorm2d(256)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(256, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.BatchNorm2d(1024),
        )
        self.out = nn.Sequential(
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
        
#         self.hidden0 = nn.Sequential(
#             nn.Linear(self.in_dim, 1024),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.3)
#         )
#         self.hidden1 = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.3)
#         )
#         self.hidden2 = nn.Sequential(
#             nn.Linear(512, 256),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.3)
#         )
#         self.out = nn.Sequential(
#             torch.nn.Linear(256, self.out_dim),
#             torch.nn.Sigmoid()
#         )
    
    def forward(self, x):
#         x = self.hidden0(x)
#         x = self.hidden1(x)
#         x = self.hidden2(x)
#         x = self.out(x)
        x = self.conv(x)
        x = self.fc_layer(x)
        x = self.out(x)
        return x

In [14]:
#Next step: make use convolutions instead of taking in flattened photo.
class GeneratorNet(torch.nn.Module):
    def __init__(self, in_dim, out_dim, input_size):
        super(GeneratorNet, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.input_size = input_size
        
        self.fc_layer = nn.Sequential(
            nn.Linear(self.in_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 128 * out_dim),
            nn.BatchNorm1d(128 * out_dim),
            nn.LeakyReLU(0.2)
        )
        
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size = 4, stride = 2, padding = 1),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, self.out_dim, kernel_size = 4, stride = 2, padding = 1),
            nn.Tanh()
        )
        
#         self.hidden0 = nn.Sequential(
#             nn.Linear(self.in_dim, 256),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden1 = nn.Sequential(
#             nn.Linear(256, 512),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden2 = nn.Sequential(
#             nn.Linear(512, 1024),
#             nn.LeakyReLU(0.2)
#         )
#         self.out = nn.Sequential(
#             nn.Linear(1024, self.out_dim),
#             nn.Tanh()
#         )
    
    def forward(self, x):
#         x = self.hidden0(x)
#         x = self.hidden1(x)
#         x = self.hidden2(x)
#         x = self.out(x)
        x = self.fc_layer(x)
        x = x.view(-1, 128, 28, 28)
        x = self.deconv(x)
        
        return x


In [15]:
#For the initial design with flattened images. Still learning how to use pytorch
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, input_size, input_size)

In [16]:
a, b = data_loader.__iter__().__next__()
# print(x)
print(b)
print(a.size())

x = a.view(-1, 28*28)
x.size()

a.shape[1]
# x = x.view(-1, 28*28)
# z = torch.rand((self.batch_size, self.z_dim))

tensor([5, 4, 4, 4, 8, 2, 9, 7, 8, 6, 9, 5, 2, 6, 7, 2, 5, 9, 3, 8, 2, 0, 8, 8,
        0, 0, 4, 7, 4, 2, 6, 8, 3, 2, 7, 8, 1, 9, 9, 3, 8, 5, 1, 7, 1, 5, 1, 5,
        7, 3, 2, 4, 8, 7, 8, 2, 4, 6, 9, 7, 0, 2, 6, 9, 8, 4, 7, 7, 0, 3, 4, 2,
        8, 4, 2, 9, 8, 4, 0, 7, 2, 6, 5, 1, 1, 6, 1, 5, 3, 0, 6, 6, 5, 5, 2, 1,
        5, 9, 3, 6])
torch.Size([100, 1, 28, 28])


1

In [17]:
class GAN(object):
    def __init__(self):
        self.epoch = 5
        self.sample_num = 100
        self.gpu_mode = True
        self.batch_size = 100
        self.z_dim = 100
        self.input_size = 100
        
        self.dataset = 'MNIST'
        self.save_dir = 'models/'
        self.result_dir = 'results/'
        self.model_name = 'BasicModel'
        
        trans = transforms.Compose(
            [transforms.Resize((self.input_size, self.input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean = (0.1307, ), std = (0.3081,))
            ])
                
        self.data_loader = torch.utils.data.DataLoader(
               datasets.MNIST(root = './data', train=True, download = True, transform = trans), 
               batch_size = self.batch_size, 
               shuffle = True)
        data = self.data_loader.__iter__().__next__()[0]
        
        self.num_batches = len(data_loader)
        
        #Change out_dim when I make networks use convolutions
        self.G = GeneratorNet(in_dim = self.z_dim, out_dim = 28*28, input_size = self.input_size)
        self.D = DiscriminatorNet(in_dim = 28*28, out_dim = 1, input_size = self.input_size)
        self.D_optimizer = optim.Adam(self.D.parameters(), lr=0.0002)
        self.G_optimizer = optim.Adam(self.G.parameters(), lr=0.0002)
        
        self.BCE_loss = nn.BCELoss()
        
        self.sample_z = torch.rand((self.batch_size, self.z_dim))
        
        if self.gpu_mode:
            self.G.cuda()
            self.D.cuda()
            self.BCE_loss = nn.BCELoss().cuda()
            self.sample_z = self.sample_z.cuda()
            
    def train(self):
        self.train_hist = {}
        self.train_hist['D_loss'] = []
        self.train_hist['G_loss'] = []
        self.train_hist['per_epoch_time'] = []
        self.train_hist['total_time'] = []
        
        self.y_real = torch.ones(self.batch_size, 1)
        self.y_fake = torch.zeros(self.batch_size, 1)
        
        if self.gpu_mode:
            self.y_real, self.y_fake = Variable(self.y_real.cuda()), Variable(self.y_fake.cuda())
            
        start_time = time.time()
        
        for epoch in range(self.epoch):
            
            self.G.train()
            epoch_start_time = time.time()
            
            for iter, (x, _) in enumerate(data_loader):
                
                x = x.view(-1, 28*28)
                z = torch.rand((self.batch_size, self.z_dim))
                
                if self.gpu_mode:
                    x, z = Variable(x.cuda()), Variable(z.cuda())
                
                #First train D on real and fake samples
                self.D_optimizer.zero_grad()
#                 D_real = self.D(images_to_vectors(x))
                D_real = self.D(x)
                D_real_loss = self.BCE_loss(D_real, self.y_real)
                
                gen_ims = self.G(z)
                D_fake = self.D(gen_ims)
                D_fake_loss = self.BCE_loss(D_fake, self.y_fake)
                
                D_loss = D_real_loss + D_fake_loss
                self.train_hist['D_loss'].append(D_loss.item())
                
                D_loss.backward()
                self.D_optimizer.step()
                
                #Now train G
                self.G_optimizer.zero_grad()
                gen_ims = self.G(z)
                D_fake = self.D(gen_ims)
                G_loss = self.BCE_loss(D_fake, self.y_real)
                
                self.train_hist['G_loss'].append(G_loss.item())
                
                G_loss.backward()
                self.G_optimizer.step()
                
                self.train_hist['per_epoch_time'].append(time.time() - epoch_start_time)
            
            self.train_hist['total_time'].append(time.time() - start_time)
            with torch.no_grad():
                self.visualize_results((epoch+1))
            print("Completed epoch {}".format(epoch+1))
                
        print("Done Training!")
        self.save()
        
        
        generate_animation('{}/{}/{}/{}'.format(self.result_dir, self.dataset, self.model_name, self.model_name),
                                 self.epoch)
        loss_plot(self.train_hist, os.path.join(self.save_dir, self.dataset, self.model_name), self.model_name)
        
    def visualize_results(self, epoch, fix=True):
        self.G.eval()
        
        if not os.path.exists(self.result_dir + '/' + self.dataset + '/' + self.model_name):
            os.makedirs(self.result_dir + '/' + self.dataset + '/' + self.model_name)
            
        tot_num_samples = min(self.sample_num, self.batch_size)
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples)))
        
        samples = self.G(self.sample_z)
        samples = vectors_to_images(samples)
        
        if self.gpu_mode:
            samples = samples.cpu().data.numpy().transpose(0, 2, 3, 1)
        else:
            samples = samples.data.numpy().transpose(0, 2, 3, 1)
            
        samples = (samples + 1)/2
        
        
        save_images(samples[:image_frame_dim * image_frame_dim, :, :, :], 
                    [image_frame_dim, image_frame_dim],
        '{}/{}/{}/{}_epoch{:03}.png'.format(self.result_dir, self.dataset, self.model_name, self.model_name, epoch))
        
    
    def save(self):
        save_dir = os.path.join(self.save_dir, self.dataset, self.model_name)
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        torch.save(self.G.state_dict(), os.path.join(save_dir, self.model_name + '_G.pkl'))
        torch.save(self.D.state_dict(), os.path.join(save_dir, self.model_name + '_D.pkl'))
        
        with open(os.path.join(save_dir, self.model_name + '_history.pkl'), 'wb') as f:
            pickle.dump(self.train_hist, f)
    
    def load(self):
        save_dir = os.path.join(self.save_dir, self.dataset, self.model_name)
        
        self.G.load_state_dict(torch.load(os.path.join(save_dir, self.model_name + '_G.pkl')))
        self.D.load_state_dict(torch.load(os.path.join(save_dir, self.model_name + '_D.pkl')))    
    

In [18]:
my_GAN = GAN()
my_GAN.train()

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at ..\aten\src\THC\THCGeneral.cpp:74

In [ ]:
my_GAN.sample_z.shape

In [37]:
my_GAN.train_hist['per_epoch_time'][-1]/60

16.68210436105728

Below stuff is OBSOLETE. Consolidated into GAN class + added the GPU integration.

Keeping for now while ensuring above is working.

In [ ]:
def noise(size):
    n = Variable(torch.randn(size, 100))
    return n

In [ ]:
loss = nn.BCELoss()

In [ ]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data

In [ ]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    optimizer.zero_grad()
    
    pred_real = discriminator(real_data)
    err_real = loss(pred_real, ones_target(N))
    err_real.backward()
    
    pred_fake = discriminator(fake_data)
    err_fake = loss(pred_fake, zeros_target(N))
    err_fake.backward()
    
    optimizer.step()
    
    return err_real + err_fake, pred_real, pred_fake

In [ ]:
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    
    optimizer.zero_grad()
    pred = discriminator(fake_data)
    err = loss(pred, ones_target(N))
    err.backward()
    
    optimizer.step()
    
    return err
    

In [ ]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [ ]:
GPU = True

if GPU:
    loss = loss.cuda()
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    test_noise = test_noise.cuda()

In [ ]:



num_epochs = 100

start = time.time()

for epoch in range(num_epochs):
    for n_batch, (real_batch, _) in enumerate(data_loader):
        N = real_batch.size(0)
        
        real_data = Variable(images_to_vectors(real_batch))
        fake_data = generator(noise(N).cuda()).detach()
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer, real_data, fake_data)
        
        fake_data = generator(noise(N))
        g_error = train_generator(g_optimizer, fake_data)
        
        if (n_batch)%100 == 0:
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data
    print("Epoch:{} \t Time: {}".format(epoch, time.time() - start))

end = time.time() - start

In [ ]:
dev = torch.device('cuda:0')

torch.cuda.current_device()

torch.cuda.get_device_name(0)

In [ ]:
discriminator.to(dev)